<a href="https://colab.research.google.com/github/finesun16/netology_pyda_27_hw/blob/master/hw7_python_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import sys
import os
import json
import requests
from pprint import pprint

In [16]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 'drive/MyDrive/Colab Notebooks/netology_pyda27_hw/libs',
 '/drive/MyDrive/Colab Notebooks/netology_pyda27_hw/libs']

In [ ]:
for root, directory, file in os.walk('drive'):
    print(root, directory, file)

In [15]:
sys.path.append('/drive/MyDrive/Colab Notebooks/netology_pyda27_hw/libs')

In [18]:
from exchange import Rate

In [19]:
Rate().usd()

71.8118

In [45]:
a = Rate('full')

In [49]:
a.usd()

{'CharCode': 'USD',
 'ID': 'R01235',
 'Name': 'Доллар США',
 'Nominal': 1,
 'NumCode': '840',
 'Previous': 71.1964,
 'Value': 71.8118}

In [51]:
a.make_format('USD')

{'CharCode': 'USD',
 'ID': 'R01235',
 'Name': 'Доллар США',
 'Nominal': 1,
 'NumCode': '840',
 'Previous': 71.1964,
 'Value': 71.8118}

## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [73]:
response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
response['AUD']


{'CharCode': 'AUD',
 'ID': 'R01010',
 'Name': 'Австралийский доллар',
 'Nominal': 1,
 'NumCode': '036',
 'Previous': 51.9093,
 'Value': 52.4298}

### 1 вариант: функция

In [44]:
def name_max_value():
    """
    получает данные с сервиса https://www.cbr-xml-daily.ru/daily_json.js
    создает словарь с ключом - название валюты и значением  - отношение значения к номиналу
    возвращает название валюты с максимальным значением 
    """
    response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute'] 
    dict_ = {}
    for currency in response:
        dict_[response[currency]['Name']]= response[currency]['Value']/response[currency]['Nominal']
    return max(dict_, key=dict_.get)

In [40]:
name_max_value()

'СДР (специальные права заимствования)'

### 2 вариант: наследование класса Rate


In [67]:
class Max_rate(Rate):
    def __init__(self):
        super().__init__(format='full')

    def currency_name(self, currency):
        """Получение имени валюты"""
        return self.make_format(currency)['Name']
    
    def currency_nominal_value(self, currency):
        """Получение курса валюты вне зависимости от номинала"""
        return self.make_format(currency)['Value']/self.make_format(currency)['Nominal']
    
    def new_data(self):
        """получение нового словаря имя валюты - курс"""
        result = {}
        for currency in self.exchange_rates():
            result[self.currency_name(currency)] = self.currency_nominal_value(currency)
        return result
    
    def max_value(self):
        """получение назнания валюты с максимальным значением курса"""
        return max(self.new_data(), key=self.new_data().get)

In [68]:
b = Max_rate()

In [70]:
b.currency_nominal_value('TRY')

7.2262

In [71]:
b.max_value()

'СДР (специальные права заимствования)'

## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

### 1 вариант
Изменение класса Rate (возьмем за основу класс Rate внесем изменения  получится новый класс RateDiff)

In [85]:
class RateDidd:
    def __init__(self, format='value', diff = False):
        self.format = format
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах вариантах:
        - полная информация о валюте при self.format = 'full', не зависит от значения self.diff:
        RateDiff('full', diff).make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        возвращает значение  'Value'
        RateDiff('value', true).make_format('EUR')
        79.4966

        возвращает измение курса по сравнению с прошлым значением. 
        RateDiff('value', diff).make_format('EUR')
        0.1799
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value' and self.diff == False:
                return response[currency]['Value']
            return response[currency]['Previous'] - response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')

    def AZN(self):
        """Возвращает курс азербайджанского маната на сегодня в формате self.format"""
        return self.make_format('AZN')

In [82]:
a = RateDidd('value')
a.eur()

82.2173

In [83]:
a = RateDidd('value', True)
a.eur()

-0.5621000000000009

In [81]:
a = RateDidd('full')
a.eur()

{'CharCode': 'EUR',
 'ID': 'R01239',
 'Name': 'Евро',
 'Nominal': 1,
 'NumCode': '978',
 'Previous': 81.6552,
 'Value': 82.2173}

### 2 вариант: 
создание нового класса путем наследования

In [117]:
class RateDf(Rate):
    def __init__(self, format, diff = False):
        super().__init__(format)
        self.diff = diff
        
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах вариантах:
        - полная информация о валюте при self.format = 'full', не зависит от значения self.diff:
        RateDf('full', diff).make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        возвращает значение  'Value'
        RateDf('value', true).make_format('EUR')
        79.4966

        возвращает измение курса по сравнению с прошлым значением. 
        RateDf('value', diff).make_format('EUR')
        0.1799
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value' and self.diff == False:
                return response[currency]['Value']
            return response[currency]['Previous'] - response[currency]['Value']
        
        return 'Error'

In [118]:
b = RateDf('value', True)
b.eur()

-0.5621000000000009

In [119]:
b = RateDf('value')
b.eur()

82.2173

In [120]:
b = RateDf('full')
b.eur()

{'CharCode': 'EUR',
 'ID': 'R01239',
 'Name': 'Евро',
 'Nominal': 1,
 'NumCode': '978',
 'Previous': 81.6552,
 'Value': 82.2173}

## Задание 3
Напишите класс **Designer**, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия.

# Система повышения сотрудников

In [124]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [ ]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [178]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        self.awards = awards

    def check_awards(self):
        self.seniority += 2 * self.awards 

    def awards_up(self):
        """Повышает количество премий сотрудника"""
        self.awards += 1
    
    def check_if_it_is_time_for_upgrade(self):
        if self.seniority == 0 or self.seniority % 4 == 0:
            self.check_awards()
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority += 1

        if self.grade % 2 == 0:
            self.awards_up()
        
        # условие повышения сотрудника из презентации
        if self.seniority % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [179]:
# проверяем как работает система повышения сотрудников на примере отдела разработки
m = Designer('Maria', 0, 2)

In [181]:
for i in range(20):
    m.check_if_it_is_time_for_upgrade()

Maria 1
Maria 1
Maria 2
Maria 2
Maria 2
Maria 2
Maria 2
Maria 3
Maria 3
Maria 3
Maria 3
Maria 3
Maria 3
Maria 3
Maria 4
Maria 4
Maria 4
Maria 4
Maria 4
Maria 5


In [109]:
# проверяем как работает система повышения сотрудников на примере отдела разработки

# разработчик Александр только что пришел в компанию
alex = Developer('Александр', 0)

In [111]:
alex.name

'Александр'

In [112]:
alex.grade

1

In [121]:
alex.seniority

0

In [ ]:
for i in range(20):
    alex.check_if_it_is_time_for_upgrade()

Александр 1
Александр 1
Александр 1
Александр 1
Александр 2
Александр 2
Александр 2
Александр 2
Александр 2
Александр 3
Александр 3
Александр 3
Александр 3
Александр 3
Александр 4
Александр 4
Александр 4
Александр 4
Александр 4
Александр 5
